In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
#     "url": "https://zen-cpd-zen.apps.icp4d-experiences-lite.demo.ibmcloud.com",

wml_credentials = {
    "instance_id": "openshift",
    "url": "https://zen-cpd-zen.apps.icp4d-experiences-lite-02.demo.ibmcloud.com",
    "username":"admin",
    "password": "password",
    "version": "2.5.0"
}
client = WatsonMachineLearningAPIClient(wml_credentials)

In [5]:
client.version

'1.0.55'

In [6]:
client.set.default_space('5dc5cf9a-d16f-445f-ad1d-83d0ef23198f')

'SUCCESS'

In [7]:
client.repository.list_models()

------------------------------------  ---------------------------  ------------------------  --------------
GUID                                  NAME                         CREATED                   TYPE
637758dd-44d5-4874-817f-438c658fccda  Mortgage Default Prediction  2019-12-12T15:46:19.002Z  wml-hybrid_0.1
------------------------------------  ---------------------------  ------------------------  --------------


In [8]:
model_details = client.repository.get_model_details(model_uid='637758dd-44d5-4874-817f-438c658fccda')

In [14]:
deployment_details = None
for resource in client.deployments.get_details()['resources']:
    if resource['entity']['name'] == 'Mortgage Default - Production':
        deployment_details = resource
deployment_details

{'metadata': {'parent': {'href': ''},
  'guid': 'fdac40b1-3281-4c9b-b540-07faf6bc2a35',
  'modified_at': '',
  'created_at': '2019-12-12T15:46:49.207Z',
  'href': '/v4/deployments/fdac40b1-3281-4c9b-b540-07faf6bc2a35'},
 'entity': {'name': 'Mortgage Default - Production',
  'custom': {},
  'online': {},
  'description': '',
  'space': {'href': '/v4/spaces/5dc5cf9a-d16f-445f-ad1d-83d0ef23198f'},
  'status': {'state': 'ready',
   'online_url': {'url': 'https://zen-cpd-zen.apps.icp4d-experiences-lite-02.demo.ibmcloud.com/v4/deployments/fdac40b1-3281-4c9b-b540-07faf6bc2a35/predictions'}},
  'asset': {'href': '/v4/models/637758dd-44d5-4874-817f-438c658fccda?space_id=5dc5cf9a-d16f-445f-ad1d-83d0ef23198f'},
  'auto_redeploy': False}}

In [15]:
import json
with open('/project_data/data_asset/mortgage_feed.json', 'r') as scoring_file:
    data = json.load(scoring_file)

In [16]:
import random
values = []
for _ in range(3):
    values.append(random.choice(data['values']))

In [17]:
scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA:
                   [
                       {
                           "fields": data['fields'],
                           "values": values
                       }
                   ]
                  }

In [18]:
scoring_payload

{'input_data': [{'fields': ['ID',
    'Income',
    'AppliedOnline',
    'Residence',
    'Yrs_at_Current_Address',
    'Yrs_with_Current_Employer',
    'Number_of_Cards',
    'Creditcard_Debt',
    'Loans',
    'Loan_Amount',
    'SalePrice',
    'Location'],
   'values': [['100790',
     44072,
     'NO',
     'Owner Occupier',
     5,
     5,
     1,
     1250,
     0,
     10590,
     275000,
     'L100'],
    ['100419',
     51219,
     'NO',
     'Private Renting',
     14,
     15,
     2,
     1292,
     1,
     7005,
     165000,
     'L111'],
    ['101764',
     44787,
     'NO',
     'Owner Occupier',
     9,
     4,
     2,
     982,
     0,
     8175,
     295000,
     'L100']]}]}

In [19]:
predictions = client.deployments.score(deployment_details['metadata']['guid'], scoring_payload)

In [20]:
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['YES', [0.1, 0.9]], ['YES', [0.1, 0.9]], ['NO', [0.9, 0.1]]]}]}